### Read files

In [1]:
import os

dataRoot = os.path.abspath("../data/")
datafileRoot = os.path.join(dataRoot, "src")
datafiles = [os.path.join(datafileRoot, path) for path in os.listdir(datafileRoot)]
labelfile = os.path.join(dataRoot, "label.csv")

labelfile, datafiles

('/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/preprocessed/demo/data/label.csv',
 ['/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/preprocessed/demo/data/src/src2.java',
  '/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/preprocessed/demo/data/src/src1.java',
  '/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/preprocessed/demo/data/src/src3.java'])

In [2]:
import pandas as pd

label_df = pd.read_csv(labelfile)
label_df.head()

,name,version,name1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.3,org.apache.tools.ant.taskdefs.ExecuteOn,11,4,2,14,42,29,2,...,1,1,0.885057,0.232323,3,4,34.545455,3,1.2727,0
1,ant,1.3,org.apache.tools.ant.DefaultLogger,14,1,1,8,32,49,4,...,1,0,0.000000,0.307692,0,0,16.857143,6,1.6429,2
2,ant,1.3,org.apache.tools.ant.taskdefs.TaskOutputStream,3,2,0,1,9,0,0,...,1,1,0.714286,0.666667,1,1,17.333333,1,0.6667,0


In [3]:
with open(datafiles[0]) as f:
    filelines = f.read()

filelines

'package org.apache.velocity.runtime;\n\n/*\n * Licensed to the Apache Software Foundation (ASF) under one\n * or more contributor license agreements.  See the NOTICE file\n * distributed with this work for additional information\n * regarding copyright ownership.  The ASF licenses this file\n * to you under the Apache License, Version 2.0 (the\n * "License"); you may not use this file except in compliance\n * with the License.  You may obtain a copy of the License at\n *\n *   http://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing,\n * software distributed under the License is distributed on an\n * "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY\n * KIND, either express or implied.  See the License for the\n * specific language governing permissions and limitations\n * under the License.    \n */\n\nimport org.apache.velocity.runtime.parser.Parser;\nimport org.apache.velocity.util.SimplePool;\n\n\n/**\n * This wraps the orig

### remove comments

In [4]:
import re

def remove_java_comments(codedata: str) -> str:
    """remove comments in the java source code"""
    codedata = re.sub(r"/\*(.*?)\*/", "", codedata, flags=re.MULTILINE | re.DOTALL)
    codedata = re.sub(r"//.*", "", codedata)
    codedata = re.sub(r"\n\n", "\n", codedata)
    return codedata

In [5]:
new_filelines = remove_java_comments(filelines)

# print(new_filelines)
new_filelines

'package org.apache.velocity.runtime;\n\nimport org.apache.velocity.runtime.parser.Parser;\nimport org.apache.velocity.util.SimplePool;\n\npublic class ParserPoolImpl implements ParserPool {\n    RuntimeServices rsvc = null;\n    SimplePool pool = null;\n    int max = RuntimeConstants.NUMBER_OF_PARSERS;\n    \n    public void initialize(RuntimeServices rsvc)\n    {\n        this.rsvc = rsvc;\n        max = rsvc.getInt(RuntimeConstants.PARSER_POOL_SIZE, RuntimeConstants.NUMBER_OF_PARSERS);\n        pool = new SimplePool(max);\n        for (int i = 0; i < max; i++)\n        {\n            pool.put(rsvc.createNewParser());\n        }\n        if (rsvc.getLog().isDebugEnabled())\n        {\n            rsvc.getLog().debug("Created \'" + max + "\' parsers.");\n        }\n    }\n    \n    public Parser get()\n    {\n        Parser parser = (Parser) pool.get();\n        if (parser == null)\n        {\n            rsvc.getLog().debug("Created new " +\n                    "parser (pool exhauste

In [6]:
datalist = []
for datafile in datafiles:
    with open(datafile) as f:
        lines = f.read()
    datalist.append(remove_java_comments(lines))

datalist

['package org.apache.velocity.runtime;\n\nimport org.apache.velocity.runtime.parser.Parser;\nimport org.apache.velocity.util.SimplePool;\n\npublic class ParserPoolImpl implements ParserPool {\n    RuntimeServices rsvc = null;\n    SimplePool pool = null;\n    int max = RuntimeConstants.NUMBER_OF_PARSERS;\n    \n    public void initialize(RuntimeServices rsvc)\n    {\n        this.rsvc = rsvc;\n        max = rsvc.getInt(RuntimeConstants.PARSER_POOL_SIZE, RuntimeConstants.NUMBER_OF_PARSERS);\n        pool = new SimplePool(max);\n        for (int i = 0; i < max; i++)\n        {\n            pool.put(rsvc.createNewParser());\n        }\n        if (rsvc.getLog().isDebugEnabled())\n        {\n            rsvc.getLog().debug("Created \'" + max + "\' parsers.");\n        }\n    }\n    \n    public Parser get()\n    {\n        Parser parser = (Parser) pool.get();\n        if (parser == null)\n        {\n            rsvc.getLog().debug("Created new " +\n                    "parser (pool exhaust

### modify dataframe

In [7]:
new_df = label_df.drop(columns='name1')
new_df['data'] = datalist

new_df

,name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug,data
0,ant,1.3,11,4,2,14,42,29,2,12,...,1,0.885057,0.232323,3,4,34.545455,3,1.2727,0,package org.apache.velocity.runtime;\n\nimport...
1,ant,1.3,14,1,1,8,32,49,4,4,...,0,0.000000,0.307692,0,0,16.857143,6,1.6429,2,\n\npackage org.apache.tools.mail;\nimport jav...
2,ant,1.3,3,2,0,1,9,0,0,1,...,1,0.714286,0.666667,1,1,17.333333,1,0.6667,0,\npackage org.apache.ivy.plugins.latest;\nimpo...


### save dataframe

In [8]:
savepath_json = os.path.join(dataRoot, 'trg', 'newdata.jsonl')
new_df.to_json(savepath_json, orient='records', lines=True)
new_df.to_json(savepath_json+'.gz', orient='records', lines=True, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

In [9]:
savepath_csv = os.path.join(dataRoot, 'trg', 'newdata.csv')
new_df.to_csv(savepath_csv, index=False)
new_df.to_csv(savepath_csv+'.gz', index=False, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

### load saved data

In [10]:
# saved_df = pd.read_csv(savepath_csv)
saved_df = pd.read_csv(savepath_csv + '.gz')
# saved_df = pd.read_json(savepath_json, lines=True)
# saved_df = pd.read_json(savepath_json + '.gz', lines=True)

saved_df.head()

,name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug,data
0,ant,1.3,11,4,2,14,42,29,2,12,...,1,0.885057,0.232323,3,4,34.545455,3,1.2727,0,package org.apache.velocity.runtime;\n\nimport...
1,ant,1.3,14,1,1,8,32,49,4,4,...,0,0.000000,0.307692,0,0,16.857143,6,1.6429,2,\n\npackage org.apache.tools.mail;\nimport jav...
2,ant,1.3,3,2,0,1,9,0,0,1,...,1,0.714286,0.666667,1,1,17.333333,1,0.6667,0,\npackage org.apache.ivy.plugins.latest;\nimpo...


---
## try to process ant-1.3

In [11]:
dataRoot = os.path.abspath("../../../PROMISE/")
sourceRoot = os.path.join(dataRoot, "source-code")
labelRoot = os.path.join(dataRoot, 'labeled-data')
prefixPath = os.path.join(dataRoot, 'resources', 'code-prefix.json')

projectName = 'ant'
versionName = 'ant-1.3'

labelpath = os.path.join(labelRoot, projectName, versionName + '.csv')

### read label csv

In [12]:
label_df = pd.read_csv(labelpath)

label_df.head()

,name,version,name1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.3,org.apache.tools.ant.taskdefs.ExecuteOn,11,4,2,14,42,29,2,...,1.0,1,0.885057,0.232323,3,4,34.545455,3,1.2727,0
1,ant,1.3,org.apache.tools.ant.DefaultLogger,14,1,1,8,32,49,4,...,1.0,0,0.000000,0.307692,0,0,16.857143,6,1.6429,2
2,ant,1.3,org.apache.tools.ant.taskdefs.TaskOutputStream,3,2,0,1,9,0,0,...,1.0,1,0.714286,0.666667,1,1,17.333333,1,0.6667,0
3,ant,1.3,org.apache.tools.ant.taskdefs.Cvs,12,3,0,12,37,32,0,...,1.0,1,0.770833,0.458333,0,0,24.083333,3,1.4167,0
4,ant,1.3,org.apache.tools.ant.taskdefs.Copyfile,6,3,0,4,21,1,0,...,1.0,0,0.880952,0.416667,2,2,21.000000,1,0.8333,0


### read one source file

In [13]:
import json

with open(prefixPath) as f:
    prefixDict = json.load(f)

prefixDict

{'ant': 'src.main',
 'camel': 'camel-core.src.main.java',
 'ivy': 'src.java',
 'jedit': '',
 'log4j': 'src.java',
 'lucene': 'src.java',
 'pbeans': 'src',
 'poi': 'src.java',
 'synapse': 'modules.core.src.main.java',
 'velocity': 'src.java',
 'xalan': 'src',
 'xerces': 'src'}

In [14]:
def read_source_file(sourceRoot, projectName, versionName, filename, postfix='.java'):
    filepath = os.path.join(sourceRoot, projectName, versionName, filename + postfix)
    lines, exist = "", False
    if os.path.exists(filepath):
        exist = True
        try:
            with open(filepath, encoding="utf-8") as f:
                lines = f.read()
        except:
            pass
    return lines, exist

In [15]:
filename = label_df.iloc[0]['name1']
if len(prefixDict[projectName]) > 0:
    filename =  "/".join(prefixDict[projectName].split('.') + filename.split('.'))
else:
    filename =  "/".join(filename.split('.'))

lines, exist = read_source_file(sourceRoot, projectName, versionName, filename)
filename, exist, lines

('src/main/org/apache/tools/ant/taskdefs/ExecuteOn',
 True,
 '/*\n * The Apache Software License, Version 1.1\n *\n * Copyright (c) 2000 The Apache Software Foundation.  All rights \n * reserved.\n *\n * Redistribution and use in source and binary forms, with or without\n * modification, are permitted provided that the following conditions\n * are met:\n *\n * 1. Redistributions of source code must retain the above copyright\n *    notice, this list of conditions and the following disclaimer. \n *\n * 2. Redistributions in binary form must reproduce the above copyright\n *    notice, this list of conditions and the following disclaimer in\n *    the documentation and/or other materials provided with the\n *    distribution.\n *\n * 3. The end-user documentation included with the redistribution, if\n *    any, must include the following acknowlegement:  \n *       "This product includes software developed by the \n *        Apache Software Foundation (http://www.apache.org/)."\n *    Al

### process all version files

In [16]:
size = label_df.shape[0]

dataList = ["" for _ in range(size)]
existList = [False for _ in range(size)]
for i in range(3):
    filename = label_df.iloc[i]['name1']
    if len(prefixDict[projectName]) > 0:
        filename =  "/".join(prefixDict[projectName].split('.') + filename.split('.'))
    else:
        filename =  "/".join(filename.split('.'))
    lines, exist = read_source_file(sourceRoot, projectName, versionName, filename)
    dataList[i] = remove_java_comments(lines)
    existList[i] = exist

df = label_df.copy()
df['data'] = dataList
df['exist'] = existList

df.head()

,name,version,name1,wmc,dit,noc,cbo,rfc,lcom,ca,...,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug,data,exist
0,ant,1.3,org.apache.tools.ant.taskdefs.ExecuteOn,11,4,2,14,42,29,2,...,0.885057,0.232323,3,4,34.545455,3,1.2727,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...,True
1,ant,1.3,org.apache.tools.ant.DefaultLogger,14,1,1,8,32,49,4,...,0.000000,0.307692,0,0,16.857143,6,1.6429,2,\npackage org.apache.tools.ant;\nimport java.i...,True
2,ant,1.3,org.apache.tools.ant.taskdefs.TaskOutputStream,3,2,0,1,9,0,0,...,0.714286,0.666667,1,1,17.333333,1,0.6667,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...,True
3,ant,1.3,org.apache.tools.ant.taskdefs.Cvs,12,3,0,12,37,32,0,...,0.770833,0.458333,0,0,24.083333,3,1.4167,0,,False
4,ant,1.3,org.apache.tools.ant.taskdefs.Copyfile,6,3,0,4,21,1,0,...,0.880952,0.416667,2,2,21.000000,1,0.8333,0,,False


In [17]:
# remove all absent data rows
df = df[df['exist']]

# remove name1 and exist columns
df = df.drop(columns=['name1', 'exist'])

df

,name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug,data
0,ant,1.3,11,4,2,14,42,29,2,12,...,1,0.885057,0.232323,3,4,34.545455,3,1.2727,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...
1,ant,1.3,14,1,1,8,32,49,4,4,...,0,0.000000,0.307692,0,0,16.857143,6,1.6429,2,\npackage org.apache.tools.ant;\nimport java.i...
2,ant,1.3,3,2,0,1,9,0,0,1,...,1,0.714286,0.666667,1,1,17.333333,1,0.6667,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...


In [18]:
def process_version(label_df, prefixDict, sourceRoot, projectName, versionName):
    size = label_df.shape[0]
    dataList = ["" for _ in range(size)]
    existList = [False for _ in range(size)]
    for i in range(size):
        filename = label_df.iloc[i]['name1']
        if len(prefixDict[projectName]) > 0:
            filename =  "/".join(prefixDict[projectName].split('.') + filename.split('.'))
        else:
            filename =  "/".join(filename.split('.'))
        lines, exist = read_source_file(sourceRoot, projectName, versionName, filename)
        dataList[i] = remove_java_comments(lines)
        existList[i] = exist

    # insert data to the dataframe
    df = label_df.copy()
    df['data'] = dataList
    df['exist'] = existList

    # remove all absent data rows
    df = df[df['exist']]
    # remove extra columns
    df = df.drop(columns=['name1', 'exist'])
    return df

In [19]:
df = process_version(label_df, prefixDict, sourceRoot, projectName, versionName)

print(df.shape) 
df.head()

(116, 24)


,name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug,data
0,ant,1.3,11,4,2,14,42,29,2,12,...,1,0.885057,0.232323,3,4,34.545455,3,1.2727,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...
1,ant,1.3,14,1,1,8,32,49,4,4,...,0,0.000000,0.307692,0,0,16.857143,6,1.6429,2,\npackage org.apache.tools.ant;\nimport java.i...
2,ant,1.3,3,2,0,1,9,0,0,1,...,1,0.714286,0.666667,1,1,17.333333,1,0.6667,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...
3,ant,1.3,12,3,0,12,37,32,0,12,...,1,0.770833,0.458333,0,0,24.083333,3,1.4167,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...
4,ant,1.3,6,3,0,4,21,1,0,4,...,0,0.880952,0.416667,2,2,21.000000,1,0.8333,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...


---
## Process Ant project

In [20]:
saveRoot = os.path.abspath("../data/trg/")

saveRoot

'/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/preprocessed/demo/data/trg'

### try one version

In [21]:
projectName = 'ant'

proj_labelRoot = os.path.join(labelRoot, projectName)

versionNames = [filename.split('.csv')[0] for filename in os.listdir(proj_labelRoot)]
labelpaths = [os.path.join(proj_labelRoot, version + '.csv') for version in versionNames]

versionNames, labelpaths

(['ant-1.6', 'ant-1.7', 'ant-1.3', 'ant-1.4', 'ant-1.5'],
 ['/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/PROMISE/labeled-data/ant/ant-1.6.csv',
  '/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/PROMISE/labeled-data/ant/ant-1.7.csv',
  '/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/PROMISE/labeled-data/ant/ant-1.3.csv',
  '/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/PROMISE/labeled-data/ant/ant-1.4.csv',
  '/data1/djp/software_defect_prediction/dataset/PROMISE-dataset/PROMISE/labeled-data/ant/ant-1.5.csv'])

In [22]:
label_df = pd.read_csv(labelpaths[0])
versionName = versionNames[0]

df = process_version(label_df, prefixDict, sourceRoot, projectName, versionName)

df.head()

,name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug,data
0,ant,1.6,3,1,0,3,19,0,0,3,...,0,0.000000,0.666667,0,0,33.000000,1,0.6667,0,\npackage org.apache.tools.ant.input;\nimport ...
1,ant,1.6,3,5,0,6,14,3,5,2,...,0,0.968750,0.555556,1,1,16.333333,3,1.6667,0,\npackage org.apache.tools.ant.types.selectors...
2,ant,1.6,10,1,0,15,39,19,4,12,...,1,0.000000,0.228571,0,0,32.600000,3,1.1000,0,\npackage org.apache.tools.ant.filters.util;\n...
3,ant,1.6,24,3,0,17,118,72,2,15,...,1,0.616667,0.185185,3,4,42.916667,6,1.7083,2,\npackage org.apache.tools.ant.taskdefs;\nimpo...
4,ant,1.6,12,1,0,3,22,44,1,2,...,1,0.000000,0.261905,0,0,11.416667,4,1.2500,0,\npackage org.apache.tools.ant.taskdefs;\nimpo...


In [23]:
saveRoot_csv = os.path.join(saveRoot, 'csv')
saveRoot_compressed = os.path.join(saveRoot, 'compressed')

if not os.path.exists(saveRoot_csv):
    os.makedirs(saveRoot_csv)
if not os.path.exists(saveRoot_compressed):
    os.makedirs(saveRoot_compressed)

In [24]:
savepath_csv = os.path.join(saveRoot_csv, versionName + '.csv')
savepath_compressed = os.path.join(saveRoot_compressed, versionName + '.csv.gz')

df.to_csv(savepath_csv, index=False)
df.to_csv(savepath_compressed, index=False, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

### all versions in project

In [25]:
def process_project(projectName, prefixDict, sourceRoot, labelRoot, saveRoot, to_compress=True):
    proj_labelRoot = os.path.join(labelRoot, projectName)

    versionNames = [filename.split('.csv')[0] for filename in os.listdir(proj_labelRoot)]
    labelpaths = [os.path.join(proj_labelRoot, version + '.csv') for version in versionNames]

    versionNum = len(versionNames)
    for i in range(versionNum):
        versionName = versionNames[i]
        label_df = pd.read_csv(labelpaths[i])
        df = process_version(label_df, prefixDict, sourceRoot, projectName, versionName)
        # save dataframe
        if to_compress:
            saveRoot_real = os.path.join(saveRoot, 'compressed')
            savepath = os.path.join(saveRoot_real, versionName + '.csv.gz')
            compression = {'method': 'gzip', 'compresslevel': 1, 'mtime': 1}
        else:
            saveRoot_real = os.path.join(saveRoot, 'csv')
            savepath = os.path.join(saveRoot_real, versionName + '.csv')
            compression = None

        if not os.path.exists(saveRoot_real):
            os.makedirs(saveRoot_real)
        df.to_csv(savepath, index=False, compression=compression)

        print(f"processed {versionName}: size={df.shape[0]}")

In [26]:
projectName = 'ant'

process_project(projectName, prefixDict, sourceRoot, labelRoot, saveRoot, to_compress=True)

processed ant-1.6: size=350


processed ant-1.7: size=741
processed ant-1.3: size=116
processed ant-1.4: size=166
processed ant-1.5: size=270


---
## process all projects

In [27]:
projectNames = os.listdir(labelRoot)

projectNames

['pbeans',
 'lucene',
 'ivy',
 'xerces',
 'xalan',
 'camel',
 'poi',
 'log4j',
 'jedit',
 'ant',
 'synapse',
 'velocity']

In [28]:
def process_all_projects(sourceRoot, labelRoot, saveRoot, prefixPath, to_compress=True):
    projectNames = os.listdir(labelRoot)
    with open(prefixPath) as f:
        prefixDict = json.load(f)
    
    for projectName in projectNames:
        process_project(projectName, prefixDict, sourceRoot, labelRoot, saveRoot, to_compress)

In [29]:
process_all_projects(sourceRoot, labelRoot, saveRoot, prefixPath, to_compress=True)

processed pbeans-1.0: size=26


processed pbeans-2.0: size=51
processed lucene-2.4: size=330
processed lucene-2.2: size=234
processed lucene-2.0: size=186
processed ivy-1.1: size=111
processed ivy-1.4: size=241
processed ivy-2.0: size=352
processed xerces-1.3: size=446
processed xerces-init: size=162
processed xerces-1.2: size=436
processed xalan-2.6: size=875
processed xalan-2.5: size=754
processed xalan-2.4: size=668
processed camel-1.4: size=473
processed camel-1.0: size=195
processed camel-1.6: size=523
processed camel-1.2: size=368
processed poi-2.5: size=380
processed poi-3.0: size=438
processed poi-2.0: size=309
processed poi-1.5: size=235
processed log4j-1.0: size=115
processed log4j-1.1: size=100
processed log4j-1.2: size=188
processed jedit-4.0: size=281
processed jedit-4.1: size=266
processed jedit-3.2: size=248
processed jedit-4.3: size=487
processed jedit-4.2: size=355
processed ant-1.6: size=350
processed ant-1.7: size=741
processed ant-1.3: size=116
processed ant-1.4: size=166
processed ant-1.5: size=2

---
## Process by spliting

In [30]:
testSplitFile = os.path.join(dataRoot, 'resources', 'test-split.json')

with open(testSplitFile) as f:
    testSplitDict = json.load(f)
testSplitDict = {k : [v] for k, v in testSplitDict.items()}

testSplitDict

{'ant': ['ant-1.7'],
 'camel': ['camel-1.6'],
 'ivy': ['ivy-2.0'],
 'jedit': ['jedit-4.3'],
 'log4j': ['log4j-1.2'],
 'lucene': ['lucene-2.4'],
 'pbeans': ['pbeans-2.0'],
 'poi': ['poi-3.0'],
 'synapse': ['synapse-1.2'],
 'velocity': ['velocity-1.6'],
 'xalan': ['xalan-2.6'],
 'xerces': ['xerces-1.3']}

In [31]:
projectNames = os.listdir(labelRoot)

trainSplitDict = {}
for projectName in projectNames:
    versionNames = [filename.split('.csv')[0] for filename in os.listdir(os.path.join(labelRoot, projectName))]
    trainSplitDict[projectName] = [version for version in versionNames if version not in testSplitDict[projectName]]

trainSplitDict

{'pbeans': ['pbeans-1.0'],
 'lucene': ['lucene-2.2', 'lucene-2.0'],
 'ivy': ['ivy-1.1', 'ivy-1.4'],
 'xerces': ['xerces-init', 'xerces-1.2'],
 'xalan': ['xalan-2.5', 'xalan-2.4'],
 'camel': ['camel-1.4', 'camel-1.0', 'camel-1.2'],
 'poi': ['poi-2.5', 'poi-2.0', 'poi-1.5'],
 'log4j': ['log4j-1.0', 'log4j-1.1'],
 'jedit': ['jedit-4.0', 'jedit-4.1', 'jedit-3.2', 'jedit-4.2'],
 'ant': ['ant-1.6', 'ant-1.3', 'ant-1.4', 'ant-1.5'],
 'synapse': ['synapse-1.1', 'synapse-1.0'],
 'velocity': ['velocity-1.4', 'velocity-1.5']}

### try to get one split dataframe

In [35]:
def process_split(splitDict, prefixDict, sourceRoot, labelRoot, shuffle=True):
    dfList = []
    for projectName, versionNames in splitDict.items():
        proj_labelRoot = os.path.join(labelRoot, projectName)
        labelpaths = [os.path.join(proj_labelRoot, version + '.csv') for version in versionNames]
        versionNum = len(versionNames)
        for i in range(versionNum):
            versionName = versionNames[i]
            label_df = pd.read_csv(labelpaths[i])
            df = process_version(label_df, prefixDict, sourceRoot, projectName, versionName)
            dfList.append(df)
    # merge dataframes
    split_df =  pd.concat(dfList)
    # shuffle data
    if shuffle:
        split_df = split_df.sample(frac=1).reset_index(drop=True)
    return split_df

In [36]:
splitDict = testSplitDict

split_df = process_split(splitDict, prefixDict, sourceRoot, labelRoot, shuffle=True)

print(split_df.shape)
split_df.head()

(4916, 24)


,name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug,data
0,camel,1.6,5,4,0,6,12,10,1,5,...,0,0.978142,0.600000,1,1,4.600000,1,0.8000,0,\npackage org.apache.camel.model;\nimport java...
1,xalan,2.6.0,9,2,0,4,24,0,0,4,...,2,0.619048,0.288889,1,6,10.777778,3,1.2222,0,\n\npackage org.apache.xalan.xsltc.dom;\nimpor...
2,xalan,2.6.0,9,2,0,9,23,36,9,9,...,0,0.529412,0.444444,0,0,8.111111,1,0.8889,0,\n\npackage org.apache.xml.dtm.ref;\nimport ja...
3,lucene,2.4,11,3,0,4,25,0,1,3,...,0,0.444444,0.287879,1,3,39.272727,6,1.5455,2,package org.apache.lucene.search;\n\nimport ja...
4,jEdit,4.3,2,1,0,2,4,1,2,0,...,0,0.000000,0.500000,0,0,26.500000,5,2.5000,0,\npackage org.gjt.sp.jedit.syntax;\nimport jav...


### process train test splits

In [45]:
def process_train_test_splits(labelRoot, testSplitFile, prefixPath, saveRoot, to_compress=True, shuffle=True):
    # load split dicts
    with open(testSplitFile) as f:
        testSplitDict = json.load(f)
    testSplitDict = {k : [v] for k, v in testSplitDict.items()}

    projectNames = os.listdir(labelRoot)
    trainSplitDict = {}
    for projectName in projectNames:
        versionNames = [filename.split('.csv')[0] for filename in os.listdir(os.path.join(labelRoot, projectName))]
        trainSplitDict[projectName] = [version for version in versionNames if version not in testSplitDict[projectName]]

    # get split dataframes
    with open(prefixPath) as f:
        prefixDict = json.load(f)
    train_split_df = process_split(trainSplitDict, prefixDict, sourceRoot, labelRoot, shuffle)
    test_split_df = process_split(testSplitDict, prefixDict, sourceRoot, labelRoot, shuffle)

    # save dataframes
    if to_compress:
        saveRoot_real = os.path.join(saveRoot, 'compressed')
        savepath_train = os.path.join(saveRoot_real, 'train-split.csv.gz')
        savepath_test = os.path.join(saveRoot_real, 'test-split.csv.gz')
        compression = {'method': 'gzip', 'compresslevel': 1, 'mtime': 1}
    else:
        saveRoot_real = os.path.join(saveRoot, 'csv')
        savepath_train = os.path.join(saveRoot_real, 'train-split.csv')
        savepath_test = os.path.join(saveRoot_real, 'test-split.csv')
        compression = None

    if not os.path.exists(saveRoot_real):
        os.makedirs(saveRoot_real)
    train_split_df.to_csv(savepath_train, index=False, compression=compression)
    test_split_df.to_csv(savepath_test, index=False, compression=compression)
    print(f"processed: size(train-split)={train_split_df.shape[0]}, size(test-split)={test_split_df.shape[0]}")

In [47]:
process_train_test_splits(labelRoot, testSplitFile, prefixPath, saveRoot, to_compress=True, shuffle=True)

processed: size(train-split)=7813, size(test-split)=4916


### load saved split dataframes

In [48]:
train_split_path = os.path.join(saveRoot, 'compressed', 'train-split.csv.gz')
test_split_path = os.path.join(saveRoot, 'compressed', 'test-split.csv.gz')

train_df = pd.read_csv(train_split_path)
test_df = pd.read_csv(test_split_path)

train_df.shape, test_df.shape

((7813, 24), (4916, 24))